In [ ]:
import cv2
import os
 
# Settings 
video_path = "C:/Users/hp/Pictures/Camera Roll/WIN_20250811_18_15_40_Pro.mp4"      
output_folder = "extracted_frames"  
frame_interval = 30  
 
os.makedirs(output_folder, exist_ok=True)
 
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()
 
frame_count = 0
saved_count = 0
 
while True:
    ret, frame = cap.read()
    if not ret:
        break 
 
  
    if frame_count % frame_interval == 0:
        filename = os.path.join(output_folder, f"frame_{saved_count:03d}.jpg")
        cv2.imwrite(filename, frame)
        saved_count += 1
 
    frame_count += 1
 
cap.release()
print(f"✅ Extracted {saved_count} frames to '{output_folder}'")
 

✅ Extracted 36 frames to 'extracted_frames'


In [5]:
import cv2
import numpy as np
import os


dataset_path = "extracted_frames" 
yunet_model = "face_detection_yunet_2023mar.onnx"
sface_model = "face_recognition_sface_2021dec.onnx"


detector = cv2.FaceDetectorYN.create(
    yunet_model,
    "",
    (320, 320),
    score_threshold=0.5, 
    
)

recognizer = cv2.FaceRecognizerSF.create(
    sface_model, ""
)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def l2_normalize(x):
    norm = np.linalg.norm(x)
    if norm == 0:
        return x
    return x / norm


for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue

    embeddings = []

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read {img_name}")
            continue

        h, w, _ = img.shape
        detector.setInputSize((w, h))

        faces = detector.detect(img)
        if faces[1] is None:
            print(f"No face detected in {img_name}")
            continue

        # Take only the first detected face
        face = faces[1][0]

        aligned_face = recognizer.alignCrop(img, face)
        face_embedding = recognizer.feature(aligned_face)

        embeddings.append(face_embedding)

    if embeddings:
        embeddings = np.array(embeddings)
        # Average embeddings and normalize
        avg_embedding = np.mean(embeddings, axis=0)
        avg_embedding = l2_normalize(avg_embedding)

        save_path = f"{person_name}.npy"
        np.save(save_path, avg_embedding)
        print(f"✅ Saved average embedding for {person_name} -> {save_path}")
    else:
        print(f"⚠ No embeddings saved for {person_name}")

✅ Saved average embedding for Arunima -> Arunima.npy
✅ Saved average embedding for ayanava -> ayanava.npy
✅ Saved average embedding for Dipasha -> Dipasha.npy
✅ Saved average embedding for nikita -> nikita.npy
✅ Saved average embedding for soham -> soham.npy


In [6]:
embedding_files = [f for f in os.listdir('.') if f.endswith('.npy')]
known_embeddings = {}
for f in embedding_files:
    name = os.path.splitext(os.path.basename(f))[0]
    emb = np.load(f)
    known_embeddings[name] = emb

print(f"Loaded embeddings for: {list(known_embeddings.keys())}")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    detector.setInputSize((w, h))

    faces = detector.detect(frame)
    faces_data = faces[1]

    if faces_data is not None:
        # Loop over all detected faces
        for face in faces_data:
            aligned_face = recognizer.alignCrop(frame, face)
            face_embedding = recognizer.feature(aligned_face)
            face_embedding = l2_normalize(face_embedding)

            best_match = None
            best_score = -1

            for name, known_emb in known_embeddings.items():
                score = cosine_similarity(face_embedding.flatten(), known_emb.flatten())
                if score > best_score:
                    best_score = score
                    best_match = name

            threshold = 0.5
            if best_score > threshold:
                label = f"{best_match} ({best_score:.2f})"
            else:
                label = "Unknown"

            x, y, w_box, h_box = map(int, face[:4])
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loaded embeddings for: ['Arunima', 'ayanava', 'Dipasha', 'nikita', 'soham']
